In [1]:
import dask.dataframe as pd
import dask

from dask.distributed import Client

client = Client(n_workers=8)

In [150]:
dl = pd.read_csv("data/DL-us-mobility-daterow.csv", dtype={'fips': 'float64'})
dl = dl[dl.admin_level == 1]

In [151]:
dl = dl.groupby(['date', 'admin1']).mean().reset_index().compute()
dl.to_csv("data/output/DL-mobility.csv")
dl.head()

,date,admin1,admin_level,fips,samples,m50,m50_index
0,2020-03-01,Alabama,1.0,1.0,133826.0,8.331,79.0
1,2020-03-01,Alaska,1.0,2.0,7782.0,0.808,20.0
2,2020-03-01,Arizona,1.0,4.0,121939.0,4.749,60.0
3,2020-03-01,Arkansas,1.0,5.0,73333.0,7.026,73.0
4,2020-03-01,California,1.0,6.0,451282.0,4.122,60.0


In [133]:
sg = pd.read_csv("data/safegraph/sd-metrics/**/*.csv.gz", compression='gzip', blocksize=None, dtype={'distance_traveled_from_home': 'float64', 'origin_census_block_group': 'string'},parse_dates=['date_range_start', 'date_range_end'])

In [135]:
sg['fips'] = sg['origin_census_block_group'].apply(lambda x: x[0:5], meta=('str'))
sg['fips_state'] = sg['origin_census_block_group'].apply(lambda x: x[0:2], meta=('str'))
sg['fips_county'] = sg['origin_census_block_group'].apply(lambda x: x[2:5], meta=('str'))
sg['date'] = sg['date_range_start'].apply(lambda x: x.date(), meta=('str'))
sg['m50'] = sg[]

,origin_census_block_group,date_range_start,date_range_end,device_count,distance_traveled_from_home,bucketed_distance_traveled,median_dwell_at_bucketed_distance_traveled,completely_home_device_count,median_home_dwell_time,bucketed_home_dwell_time,at_home_by_each_hour,part_time_work_behavior_devices,full_time_work_behavior_devices,fips,fips_state,fips_county,date
0,010150007002,2020-01-01 00:00:00-06:00,2020-01-02 00:00:00-06:00,80,5431.0,"{""16001-50000"":2,"">50000"":8,""<1000"":15,""2001-8...","{""16001-50000"":20,"">50000"":17,""<1000"":25,""2001...",25,752,"{""721-1080"":16,""361-720"":9,""61-360"":11,""<60"":1...","[48,47,49,50,47,46,43,39,38,33,34,34,34,32,31,...",5,6,01015,01,015,2020-01-01
1,010299598001,2020-01-01 00:00:00-05:00,2020-01-02 00:00:00-05:00,156,15016.0,"{""16001-50000"":34,"">50000"":17,""<1000"":14,""2001...","{""16001-50000"":49,"">50000"":41,""<1000"":51,""2001...",39,797,"{""721-1080"":31,""361-720"":30,""61-360"":10,""<60"":...","[103,100,103,101,96,89,81,72,70,67,61,62,58,56...",10,17,01029,01,029,2020-01-01
2,010299598001,2020-01-01 00:00:00-06:00,2020-01-02 00:00:00-06:00,156,15016.0,"{""16001-50000"":34,"">50000"":17,""<1000"":14,""2001...","{""16001-50000"":49,"">50000"":41,""<1000"":51,""2001...",39,797,"{""721-1080"":31,""361-720"":30,""61-360"":10,""<60"":...","[103,100,103,101,96,89,81,72,70,67,61,62,58,56...",10,17,01029,01,029,2020-01-01
3,010730109006,2020-01-01 00:00:00-06:00,2020-01-02 00:00:00-06:00,38,7419.0,"{""16001-50000"":2,"">50000"":2,""<1000"":1,""2001-80...","{""16001-50000"":7,"">50000"":13,""<1000"":1221,""200...",14,713,"{""721-1080"":5,""361-720"":3,""61-360"":4,""<60"":10,...","[18,19,17,17,18,17,20,21,17,16,13,20,17,16,14,...",2,6,01073,01,073,2020-01-01
4,011250103023,2020-01-01 00:00:00-06:00,2020-01-02 00:00:00-06:00,131,11979.0,"{""16001-50000"":15,"">50000"":20,""<1000"":12,""2001...","{""16001-50000"":54,"">50000"":136,""<1000"":58,""200...",30,750,"{""721-1080"":22,""361-720"":16,""61-360"":10,""<60"":...","[70,70,67,71,68,69,61,56,51,52,45,38,46,48,44,...",13,31,01125,01,125,2020-01-01


In [138]:
state_distance_traveled, county_distance_traveled = dask.compute(sg.groupby(['fips_state', 'date']).mean().reset_index(), 
                                                                 sg.groupby(['fips', 'date']).mean().reset_index())

In [136]:
county_distance_traveled.to_csv("data/output/County-SG-mobility.csv")
state_distance_traveled.to_csv("data/output/State-SG-mobility.csv")
state_distance_traveled
county_distance_traveled.plot(x='date', y='distance_traveled_from_home')

distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker


,fips_state,date,device_count,distance_traveled_from_home,completely_home_device_count,median_home_dwell_time,part_time_work_behavior_devices,full_time_work_behavior_devices
0,01,2020-01-01,120.158399,1.252323e+04,35.953293,762.402959,8.408471,18.556426
1,02,2020-01-01,61.329524,7.018448e+04,21.550476,590.782857,1.731429,7.853333
2,04,2020-01-01,89.494476,1.342844e+04,29.591258,759.750240,2.936840,10.152738
3,05,2020-01-01,105.093838,1.150128e+04,31.065826,749.987395,5.638189,15.105509
4,06,2020-01-01,74.046872,1.317016e+04,25.022182,754.568383,2.944826,8.856272
...,...,...,...,...,...,...,...,...
6381,56,2020-04-24,62.575610,1.043842e+04,19.453659,655.446341,4.843902,10.736585
6382,66,2020-04-24,7.884615,1.168019e+06,2.230769,589.076923,1.115385,1.538462
6383,69,2020-04-24,7.222222,4.230333e+03,2.111111,553.444444,1.000000,1.000000
6384,72,2020-04-24,40.842168,6.381459e+03,22.345556,811.292945,2.053009,2.903946
